In [3]:
%pip install pandas
%pip install requests
%pip install RateMyProfessorAPI==1.3.6

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import requests
import json
import time
import ratemyprofessor

In [5]:
# Dataframes for gpas of course sections - taken from prof. Wade's datasets
sp2020 = pd.read_csv('raw-data/sp2020.csv')
sp2021 = pd.read_csv('raw-data/sp2021.csv')
sp2022 = pd.read_csv('raw-data/sp2022.csv')
sp2023 = pd.read_csv('raw-data/sp2023.csv')
fa2020 = pd.read_csv('raw-data/fa2020.csv')
fa2021 = pd.read_csv('raw-data/fa2021.csv')
fa2022 = pd.read_csv('raw-data/fa2022.csv')



In [6]:
# For now, we are just concatenating these DF's. For the website, we'll run a for loop to collect datasets from the last 5 years
df = pd.concat([sp2020, sp2021, sp2022, sp2023, fa2020, fa2021, fa2022], ignore_index=True)
df

,Subject,Course,CRN,Course Title,Sched Type,A+,A,A-,B+,B,...,D+,D,D-,F,W,Average Grade,Primary Instructor,Section,Term,Course Section
0,AAS,100,30107,Intro Asian American Studies,DIS,12,12,1,0,2,...,0,0,0,1,0,3.68,"Boonsripaisal, Simon",NaN,NaN,NaN
1,AAS,100,41729,Intro Asian American Studies,DIS,5,12,4,1,4,...,0,0,0,1,0,3.51,"Boonsripaisal, Simon",NaN,NaN,NaN
2,AAS,100,43832,Intro Asian American Studies,DIS,0,8,7,3,6,...,0,0,0,0,0,3.28,"Uhall, Michael B",NaN,NaN,NaN
3,AAS,100,48232,Intro Asian American Studies,DIS,8,6,2,0,4,...,0,0,0,0,0,3.44,"Uhall, Michael B",NaN,NaN,NaN
4,AAS,100,48234,Intro Asian American Studies,DIS,5,4,3,1,4,...,1,0,1,1,0,3.00,"Kang, Yoonjung",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16506,VM,605,56271,Pathobiology I,LEC,0,37,0,0,59,...,0,2,0,0,0,2.98,"Zuckermann, Federico A",NaN,120228.0,AL1
16507,VM,607,64924,Pathobiology II,LEC,0,56,0,0,67,...,0,1,0,0,0,3.34,"Maddox, Carol W",NaN,120228.0,AL1
16508,VM,652,75872,LA Medicine and Surgery I,LEC,0,15,0,0,59,...,0,11,0,0,0,2.60,"Ridgway, Marcella D",NaN,120228.0,AL1
16509,VM,653,75873,SA Medicine and Surgery II,LEC,0,20,0,0,54,...,0,17,0,2,0,2.56,"Ridgway, Marcella D",NaN,120228.0,AL1


In [27]:
subject = 'AAS'
course = 100
# How to get DF of only rows with specificed subject and course number:
AAS_sections = df[(df['Subject'] == subject) & (df['Course'] == course)]
print(AAS_sections)

      Subject  Course    CRN                  Course Title Sched Type  A+   A  \
0         AAS     100  30107  Intro Asian American Studies        DIS  12  12   
1         AAS     100  41729  Intro Asian American Studies        DIS   5  12   
2         AAS     100  43832  Intro Asian American Studies        DIS   0   8   
3         AAS     100  48232  Intro Asian American Studies        DIS   8   6   
4         AAS     100  48234  Intro Asian American Studies        DIS   5   4   
5         AAS     100  50105  Intro Asian American Studies        DIS   3   6   
6         AAS     100  54572  Intro Asian American Studies        DIS   3  14   
7         AAS     100  51886  Intro Asian American Studies        DIS   4  15   
8         AAS     100  70761  Intro Asian American Studies        LCD   3   4   
9         AAS     100  70762  Intro Asian American Studies        LCD   0  17   
2098      AAS     100  30107  Intro Asian American Studies         OD  14   9   
2099      AAS     100  41729

In [7]:
# For an average GPA DF, we can collect values of certain columns as shown:
course_titles = AAS_sections['Course Title']
print(course_titles)

NameError: name 'AAS_sections' is not defined

In [8]:
# How to remove columns from DF:
columns_to_keep = ['Subject', 'Course', 'Course Title', 'Average Grade']
edited_df = df[columns_to_keep]

# Because courses are split up by sections and their respective average GPAs, we need to take the average of these GPAs as well.
# This line groups rows if they have the same subject,  course, and course title and averages the GPAs of those rows
edited_df = edited_df.groupby(['Subject', 'Course', 'Course Title'], as_index=False).agg({'Average Grade': 'mean'})
print(edited_df)

     Subject  Course                    Course Title  Average Grade
0        AAS     100    Intro Asian American Studies       3.491167
1        AAS     105        Intro to Arab Am Studies       3.700000
2        AAS     120   Intro to Asian Am Pop Culture       3.525000
3        AAS     200            U.S. Race and Empire       3.477500
4        AAS     215    US Citizenship Comparatively       3.607500
...      ...     ...                             ...            ...
3470      VM     653      SA Medicine and Surgery II       2.475000
3471      VM     654      LA Medicine and Surgery II       2.895000
3472      VM     655     SA Medicine and Surgery III       2.925000
3473      VM     656     LA Medicine and Surgery III       2.865000
3474    YDSH     320  Lit Responses to the Holocaust       3.200000

[3475 rows x 4 columns]


In [41]:
# Dataframe of gened courses. Only important data includes course, course title, and if it fulfills a certain gened requirement out of the 6
gen_ed_df = pd.read_csv('raw-data/gened-courses.csv')
gen_ed_df

,Year,Term,TermYear,Course,Course Title,ACP,CS,HUM,NAT,QR,SBS
0,2023,Spring,2023-sp,AAS 100,Intro Asian American Studies,NaN,US,NaN,NaN,NaN,SS
1,2023,Spring,2023-sp,AAS 105,Introduction to Arab American Studies,NaN,US,NaN,NaN,NaN,NaN
2,2023,Spring,2023-sp,AAS 120,Intro to Asian Am Pop Culture,NaN,US,NaN,NaN,NaN,NaN
3,2023,Spring,2023-sp,AAS 200,U.S. Race and Empire,NaN,US,HP,NaN,NaN,NaN
4,2023,Spring,2023-sp,AAS 201,US Racial & Ethnic Politics,NaN,US,NaN,NaN,NaN,SS
...,...,...,...,...,...,...,...,...,...,...,...
1055,2023,Spring,2023-sp,UP 316,Urban Informatics II,NaN,NaN,NaN,NaN,QR2,NaN
1056,2023,Spring,2023-sp,UP 330,The Modern American City,NaN,US,NaN,NaN,NaN,NaN
1057,2023,Spring,2023-sp,WRIT 303,Writing Across Media,ACP,NaN,NaN,NaN,NaN,NaN
1058,2023,Spring,2023-sp,YDSH 220,Jewish Storytelling,NaN,WCC,LA,NaN,NaN,NaN


In [57]:
# This merges the final average gpa DF and the gened DF so we can see a gened course, what it fulfills, and it's average GPA in one DF
merged_df = pd.merge(gen_ed_df, edited_df, on='Course Title', how='left')
print(merged_df)

      Year    Term TermYear  Course_x                           Course Title  \
0     2023  Spring  2023-sp   AAS 100           Intro Asian American Studies   
1     2023  Spring  2023-sp   AAS 100           Intro Asian American Studies   
2     2023  Spring  2023-sp   AAS 100           Intro Asian American Studies   
3     2023  Spring  2023-sp   AAS 105  Introduction to Arab American Studies   
4     2023  Spring  2023-sp   AAS 120          Intro to Asian Am Pop Culture   
...    ...     ...      ...       ...                                    ...   
4076  2023  Spring  2023-sp    UP 316                   Urban Informatics II   
4077  2023  Spring  2023-sp    UP 330               The Modern American City   
4078  2023  Spring  2023-sp  WRIT 303                   Writing Across Media   
4079  2023  Spring  2023-sp  YDSH 220                    Jewish Storytelling   
4080  2023  Spring  2023-sp  YDSH 320             Responses to the Holocaust   

      ACP   CS  HUM  NAT   QR  SBS Subj

In [32]:
# This function takes the professor name given, in the format of the names given in the course datasets, and finds the prof's rating
school_name = "University Of Illinois at Urbana Champaign"
school = ratemyprofessor.get_school_by_name(school_name)
professor_cache = {}

def get_professor_rating(professor_name):
    if professor_name in professor_cache:
        return professor_cache[professor_name]

    if isinstance(professor_name, str):
        name_parts = professor_name.split(',')
        if len(name_parts) >= 2:
            last_name = name_parts[0].strip()
            first_name = name_parts[1].strip().split()[0]
            name = first_name + " " + last_name
        else:
            professor_cache[professor_name] = None
            return None
    else:
        professor_cache[professor_name] = None
        return None

    if school is None:
        print(f"School '{school_name}' not found.")
        return None

    try:
        professor = ratemyprofessor.get_professor_by_school_and_name(school, name)
        if professor is not None:
            rating = professor.rating
            professor_cache[professor_name] = rating
            return rating
        else:
            professor_cache[professor_name] = None
            return None
    except Exception as e:
        print(f"Error occurred for professor {professor_name}: {str(e)}")
        professor_cache[professor_name] = None
        return None

In [33]:
# Test using a single professor
fun_test = get_professor_rating("Lee, Sang")
print(fun_test)

3.5


In [34]:
remove_values = ['DIS', 'IND', 'INT', 'OD']
mask = ~df['Sched Type'].isin(remove_values)
edited_result = df.loc[mask]
columns_to_keep = ['Subject', 'Course', 'Course Title', 'Average Grade', 'Primary Instructor']
edited_df = edited_result[columns_to_keep]
edited_df

,Subject,Course,Course Title,Average Grade,Primary Instructor
8,AAS,100,Intro Asian American Studies,3.15,"Thomas, Merin A"
9,AAS,100,Intro Asian American Studies,3.53,"Lee, Sang S"
10,AAS,215,US Citizenship Comparatively,3.69,"Compoc, Kimberly M"
11,AAS,258,Muslims in America,3.91,"Rana, Junaid"
12,ABE,225,ABE Principles: Bioenvironment,3.09,"Zhang, Yuanhui"
...,...,...,...,...,...
16506,VM,605,Pathobiology I,2.98,"Zuckermann, Federico A"
16507,VM,607,Pathobiology II,3.34,"Maddox, Carol W"
16508,VM,652,LA Medicine and Surgery I,2.60,"Ridgway, Marcella D"
16509,VM,653,SA Medicine and Surgery II,2.56,"Ridgway, Marcella D"


In [35]:
#edited_df = edited_df.iloc[:100]
edited_df.to_csv('dataset.csv', index=False)

In [36]:
new_df = edited_df.copy()
chunk_size = 100  # Process the DataFrame in chunks of 100 rows

# Initialize an empty list to store the processed chunks
processed_chunks = []

# Iterate over the DataFrame in chunks
for chunk in pd.read_csv('dataset.csv', chunksize=chunk_size):
    # Create a copy of the chunk
    new_chunk = chunk.copy()
    
    # Apply the get_professor_rating function to the "Primary Instructor" column
    new_chunk["Professor Rating"] = new_chunk["Primary Instructor"].apply(lambda x: get_professor_rating(x))
    
    # Append the processed chunk to the list
    processed_chunks.append(new_chunk)

# Concatenate the processed chunks into a final DataFrame
final_df = pd.concat(processed_chunks, ignore_index=True)

In [37]:
final_df.to_csv('final_output.csv', index=False)

In [58]:
columns_to_keep = ['Course Title', 'ACP', 'CS', 'HUM', 'NAT', 'QR', 'SBS']
edited_gen_ed_df = gen_ed_df[columns_to_keep]
merged_df = pd.merge(edited_gen_ed_df, edited_df, on='Course Title', how='left')
merged_df = merged_df[['Subject', 'Course', 'Course Title', 'Average Grade', 'Primary Instructor', 'ACP', 'CS', 'HUM', 'NAT', 'QR', 'SBS']]
merged_df

,Subject,Course,Course Title,Average Grade,Primary Instructor,ACP,CS,HUM,NAT,QR,SBS
0,AAS,100.0,Intro Asian American Studies,3.15,"Thomas, Merin A",NaN,US,NaN,NaN,NaN,SS
1,AAS,100.0,Intro Asian American Studies,3.53,"Lee, Sang S",NaN,US,NaN,NaN,NaN,SS
2,AAS,100.0,Intro Asian American Studies,3.41,"Lee, Sang S",NaN,US,NaN,NaN,NaN,SS
3,NaN,NaN,Introduction to Arab American Studies,NaN,NaN,NaN,US,NaN,NaN,NaN,NaN
4,AAS,120.0,Intro to Asian Am Pop Culture,3.62,"Tabares, Leland",NaN,US,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4076,UP,316.0,Urban Informatics II,3.26,"Lee, Bumsoo",NaN,NaN,NaN,NaN,QR2,NaN
4077,UP,330.0,The Modern American City,2.52,"Yenigun, Ozge",NaN,US,NaN,NaN,NaN,NaN
4078,NaN,NaN,Writing Across Media,NaN,NaN,ACP,NaN,NaN,NaN,NaN,NaN
4079,NaN,NaN,Jewish Storytelling,NaN,NaN,NaN,WCC,LA,NaN,NaN,NaN


In [ ]:
merged_df.to_csv('gened_output.csv', index=False)

In [9]:
unique_subjects = edited_df['Subject'].unique()
unique_df = pd.DataFrame({'Subject': unique_subjects})
unique_df

,Subject
0,AAS
1,ABE
2,ACCY
3,ACE
4,ACES
...,...
150,UKR
151,UP
152,VCM
153,VM


In [11]:
unique_df.to_csv('subjects.csv', index=False)